<a href="https://colab.research.google.com/github/soerenetler/spacy_target_answer/blob/master/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
os.system("git clone https://github.com/soerenetler/squad-question-generation.git")
os.system("git clone https://github.com/soerenetler/spacy_target_answer.git")
os.system("git clone https://github.com/soerenetler/crf_target_answer.git")

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!(cd spacy_target_answer; git pull)
!(cd squad-question-generation/; git pull)

Already up to date.
Already up to date.


In [5]:
!cp crf_target_answer/Custom.py Custom.py

In [6]:
!nvidia-smi

Wed Apr 28 10:11:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
#!pip install --quiet -U pip setuptools wheel

In [8]:
!pip install --quiet -U spacy[cuda110,transformers,lookups]

     |████████████████████████████████| 12.8MB 7.0MB/s 
     |████████████████████████████████| 1.1MB 52.0MB/s 
     |████████████████████████████████| 460kB 54.1MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 9.1MB 57.3MB/s 
     |████████████████████████████████| 165.3MB 100kB/s 
     |████████████████████████████████| 93.4MB 53kB/s 
     |████████████████████████████████| 122kB 62.7MB/s 
     |████████████████████████████████| 2.1MB 53.8MB/s 
     |████████████████████████████████| 1.0MB 51.3MB/s 
     |████████████████████████████████| 901kB 51.2MB/s 
     |████████████████████████████████| 3.3MB 48.8MB/s 


In [9]:
!python -m spacy download en_core_web_sm --quiet
!python -m spacy download en_core_web_trf --quiet
!python -m spacy download en_core_web_lg --quiet

2021-04-28 10:12:21.719766: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.7MB 8.3MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2021-04-28 10:12:29.476343: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 459.7MB 39kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
2021-04-28 10:13:08.511054: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 778.8MB 23kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [13]:
!spacy validate

2021-04-28 10:15:40.727589: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.0.6) =================
ℹ spaCy installation: /usr/local/lib/python3.7/dist-packages/spacy

NAME              SPACY            VERSION                            
en_core_web_lg    >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_trf   >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_sm    >=3.0.0,<3.1.0   3.0.0   ✔



In [9]:
# Train NER
!python -m spacy project run corpus spacy_target_answer

2021-04-26 19:46:40.104800: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

=================================== corpus ===================================
Running command: /usr/bin/python3 -m spacy convert /content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/quac/sentences_conll corpus -c ner
2021-04-26 19:46:45.057616: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (2302 documents): corpus/dev.spacy
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1000 documents): corpus/test.spacy
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (9263 documents): corpus/train.spacy


In [13]:
!python -m spacy init fill-config ./spacy_target_answer/configs/base_config.cfg ./spacy_target_answer/configs/config.cfg
!python -m spacy project run train spacy_target_answer

2021-04-26 07:03:00.127155: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
spacy_target_answer/configs/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
2021-04-26 07:03:04.039780: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

=================================== train ===================================
Running command: /usr/bin/python3 -m spacy train configs/config.cfg -o training/ --gpu-id 0 --paths.train corpus/train.spacy --paths.dev corpus/dev.spacy --output /content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/training
2021-04-26 07:03:06.832328: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

In [14]:
!python -m spacy project run evaluate spacy_target_answer

2021-04-26 13:43:10.899696: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

================================== evaluate ==================================
Running command: /usr/bin/python3 -m spacy evaluate /content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/training/model-best ./corpus/test.spacy --output /content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/metrics/config.json --gpu-id 0
2021-04-26 13:43:15.888179: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

================================== Results ==================================

TOK     -    
NER P   50.37
NER R   6.37 
NER F   11.31
SPEED   978  


=============================== NER (per type) ===============================

             P      R       F
Answer   50.37   6.37   11.31

✔ Saved results to
/content/gdrive/MyDrive/mt-qg

In [7]:
import spacy
nlp = spacy.load('/content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/training/model-best')
doc = nlp(u"There are no major roads through Golm, only state road (Landesstraße) L 902 passes through the northern parts of the village area and offers the shortest connection to federal highway (Bundesstraße) 273.  The latter can also be reached following a road extending east from Golm past Sanssouci Park to the Jägervorstadt quarter of Potsdam. The railway between Wildpark and Nauen was opened in 1902 as part of the single-tracked bypass railway (Umgehungsbahn) intended to connect the radial railway lines originating from Berlin. [...]")
spacy.displacy.render(doc, style='ent', jupyter =True)

In [8]:
nlp2 = spacy.load('/content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/squad/training/model-best')
doc = nlp2(u"There are no major roads through Golm, only state road (Landesstraße) L 902 passes through the northern parts of the village area and offers the shortest connection to federal highway (Bundesstraße) 273.  The latter can also be reached following a road extending east from Golm past Sanssouci Park to the Jägervorstadt quarter of Potsdam. The railway between Wildpark and Nauen was opened in 1902 as part of the single-tracked bypass railway (Umgehungsbahn) intended to connect the radial railway lines originating from Berlin.  [...]")
spacy.displacy.render(doc, style='ent', jupyter =True)

In [12]:
pip install scikit-learn sklearn_crfsuite -U

     |████████████████████████████████| 22.3MB 1.7MB/s 
     |████████████████████████████████| 747kB 53.2MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [13]:
import pandas as pd
import spacy
from spacy.tokens import Doc
from tqdm import tqdm
import ast
from Custom import *
import re

spacy.require_gpu()

df = pd.read_csv("/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/quac/paragraph/test.csv")
df["askable_tokens"] = [ast.literal_eval(t) for t in df["askable_tokens"]]
df["paragraph_tokens"] = [ast.literal_eval(t) for t in df["paragraph_tokens"]]

print(df.columns)

nlp = spacy.load('/content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/training/model-best')
def custom_tokenizer(nlp):
    return lambda text : Doc(nlp.vocab, words=text)

nlp.tokenizer = custom_tokenizer(nlp)
nlp.add_pipe("sentencizer")


pred_y = []
true_y = []

with open("./trf_results.txt", "a") as file:
  for doc, y in tqdm(zip(nlp.pipe(df["paragraph_tokens"]), io2iob(df["askable_tokens"]))):
    labels = []
    for sent in doc.sents:
      for token, y_ in zip(sent, y[sent.start:sent.end]):
        if token.ent_iob_ =="O":
          labels.append("O")
          label_ = "O"
        else:
          labels.append(token.ent_iob_ + "-" + token.ent_type_)
          label_ = token.ent_iob_ + "-" + token.ent_type_
        file.write("{} {} {}\n".format(token.text, y_, label_))
      pred_y.append(labels)
      true_y.append(y) 
      file.write("\n")

Index(['Unnamed: 0', 'text_title', 'paragraph_text', 'paragraph_tokens',
       'askable_tokens'],
      dtype='object')


1000it [01:15, 13.27it/s]


In [14]:
from sklearn_crfsuite import metrics
metrics.flat_classification_report(io2iob(true_y), pred_y, digits=5)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


'              precision    recall  f1-score   support\n\n    B-Answer    0.53055   0.10499   0.17529    135815\n    I-Answer    0.61101   0.09050   0.15765   4177401\n           O    0.69534   0.97306   0.81108   9182586\n\n    accuracy                        0.69114  13495802\n   macro avg    0.61230   0.38952   0.38134  13495802\nweighted avg    0.66758   0.69114   0.60242  13495802\n'